# Chatbot
Nutzt über RAG-Pipeline die Wahlprogramme der Parteien für die Europawahl 2024, um Fragen zu beantworten.

## Imports und Setup

Im ersten Teil des Notebooks werden die benötigten Bibliotheken importiert und die RAG-Pipeline konfiguriert. Diese kann Text-, Markdown- und PDF-Dateien verarbeiten und in einem ChromaDocumentStore speichern. _Text und Markdown-Dateien sind nicht unbedingt benötigt, da alle Wahlprogramme als PDF vorliegen._ 

### Setup RAG-Pipeline
Installing dependencies:
```bash
pip install haystack-ai
pip install "sentence-transformers>=2.2.0" "huggingface_hub>=0.22.0"
pip install markdown-it-py mdit_plain pypdf
pip install chroma-haystack
pip install sentencepiece
```
`sentencepiece` wird für `T-Systems-onsite/cross-en-de-roberta-sentence-transformer` benötigt.

In [7]:
from haystack import Pipeline
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.joiners import DocumentJoiner
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = "T-Systems-onsite/cross-en-de-roberta-sentence-transformer"

# Initialisiere ChromaDocumentStore mit lokalem Speicherort
document_store = ChromaDocumentStore(
    # persist_path="./db" # Verzeichnis zum Speichern der Datenbank
)

file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
markdown_converter = MarkdownToDocument()
pdf_converter = PyPDFToDocument()
document_joiner = DocumentJoiner()

document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=200, split_overlap=50)

document_embedder = SentenceTransformersDocumentEmbedder(model=embedding_model)

# Konfiguriere DocumentWriter, um bereits vorhandene Dokumente zu überspringen
document_writer = DocumentWriter(document_store, policy=DuplicatePolicy.SKIP)

preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=markdown_converter, name="markdown_converter")
preprocessing_pipeline.add_component(instance=pdf_converter, name="pypdf_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("file_type_router.application/pdf", "pypdf_converter.sources")
preprocessing_pipeline.connect("file_type_router.text/markdown", "markdown_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("pypdf_converter", "document_joiner")
preprocessing_pipeline.connect("markdown_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - markdown_converter: MarkdownToDocument
  - pypdf_converter: PyPDFToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - file_type_router.application/pdf -> pypdf_converter.sources (List[Path])
  - file_type_router.text/markdown -> markdown_converter.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - markdown_converter.documents -> document_joiner.documents (List[Document])
  - pypdf_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (List[Document])
  - document_cleaner.documents -> document_splitter.docume

## Laden der Wahlprogramme

Im folgenden Abschnitt, weren die Wahlprogramme in `./data/election-programs` in die Preprocessing-Pipeline geladen.
Es werden jedem Dokument Metadaten in Form eines Dictionaries hinzugefügt, die den Namen des Dokuments und den Namen der Partei enthalten. Dadurch können die Dokumente durch das LLM besser den Parteien zugeordnet werden.

In [8]:
from pathlib import Path


# Pfade zu den Wahlprogrammen
def get_all_files(path: Path) -> list[Path]:
    return [file for file in path.iterdir() if file.is_file()]


def get_metadata_for_filetype_in_path(path: Path, file_type: str) -> list[dict]:
    metadata = []
    for file in path.iterdir():
        if file.is_file() and file.suffix == file_type or file.suffix == "." + file_type:
            filename = file.name
            party_name = filename.split("_")[0]
            metadata.append(
                {
                    "name": filename,
                    "partei": party_name
                }
            )

    return metadata


data_dir = Path("data/election-programs")
files = get_all_files(data_dir)

## get list of dicts of filenames and party names by document type

preprocessing_pipeline.run(
    {
        "file_type_router": {"sources": files},
        "text_file_converter": {"meta": get_metadata_for_filetype_in_path(data_dir, "txt")},
        "markdown_converter": {"meta": get_metadata_for_filetype_in_path(data_dir, "md")},
        "pypdf_converter": {"meta": get_metadata_for_filetype_in_path(data_dir, "pdf")}
    },
    document_store
)

Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 19.84it/s]


Batches:   0%|          | 0/62 [00:00<?, ?it/s]

Add of existing embedding ID: 6a2905cd2878e0b5e5ace1fc8b47af82cedb1f84b82bc29e98d685da8ffcbbed
Insert of existing embedding ID: 6a2905cd2878e0b5e5ace1fc8b47af82cedb1f84b82bc29e98d685da8ffcbbed
Add of existing embedding ID: db3c5fdcc4f1550e2a1b5a849843d192a1ea873960f3881d600b3d08c59496b9
Insert of existing embedding ID: db3c5fdcc4f1550e2a1b5a849843d192a1ea873960f3881d600b3d08c59496b9
Add of existing embedding ID: 860f457035ca6e4977612e2972a6d166f0dd7a000c839db3cfff41c203f3f79d
Insert of existing embedding ID: 860f457035ca6e4977612e2972a6d166f0dd7a000c839db3cfff41c203f3f79d
Add of existing embedding ID: dbe383d406a135342a01ac351d07fc5170b2f8161c8d6c1451f6011a57d8b814
Insert of existing embedding ID: dbe383d406a135342a01ac351d07fc5170b2f8161c8d6c1451f6011a57d8b814
Add of existing embedding ID: d30021f0a0eb6bc9b26aacc2a65f03485e5c518a1ce7ba27e72b48dc49902aa3
Insert of existing embedding ID: d30021f0a0eb6bc9b26aacc2a65f03485e5c518a1ce7ba27e72b48dc49902aa3
Add of existing embedding ID: 1c918

{'document_writer': {'documents_written': 1974}}

### Setup LLM
needed for GGUF models:
```bash
pip install llama-cpp-haystack
```

In diesem Bock wird das Modell `TheBloke/SauerkrautLM-Mixtral-8x7B-GGUF` von HuggingFace geladen und in einem Haystack-Generator verwendet, um Antworten auf die Fragen zu generieren.
Die Wahl von llama.cpp als Generator wurde getroffen, da im voraus quantisierte Modelle verwendet werden können, was gut für die Performance und den Speicherverbrauch ist. Mistral 7B kann schnell eingebunden werden, wenn das Notebook auf einem Rechner mit weniger als 24 GB VRAM läuft.

In [9]:
from haystack_integrations.components.generators.llama_cpp import LlamaCppGenerator
from huggingface_hub import hf_hub_download

cache_dir = "./.cache"

# set HF_API_TOKEN environment variable
# download the model from the Hugging Face Hub and save it to the local cache
"""
repo_name = "TheBloke/Mistral-7B-v0.1-GGUF"
file_name = "mistral-7b-v0.1.Q8_0.gguf"
model_dir = hf_hub_download(repo_id=repo_name, filename=file_name, cache_dir=cache_dir)
"""

m8x7b_repo_name = "TheBloke/SauerkrautLM-Mixtral-8x7B-GGUF"
m8x7b_file_name = "sauerkrautlm-mixtral-8x7b.Q3_K_M.gguf"
m8x7b_model_dir = hf_hub_download(repo_id=m8x7b_repo_name, filename=m8x7b_file_name, cache_dir=cache_dir)

generator = LlamaCppGenerator(
    model=m8x7b_model_dir,
    n_ctx=16_384,
    n_batch=128,
    model_kwargs={"n_gpu_layers": -1},
    generation_kwargs={"max_tokens": 750, "temperature": 0.4}
)

## Setup Chatbot

Im kommenden Abschnitt wird die Pipeline für den Chatbot konfiguriert. Es gibt zwei mögliche Templates für die PromptBuilder-Komponente, abhängig vom oben verwendeten Modell. (In beiden Fällen werden zusätzliche Metadaten zu den gefundenen Textauszügen hinzugefügt, um die Parteien zu identifizieren.)

In [10]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack.components.builders import PromptBuilder

# https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF#prompt-template-none
mistral7b_template = """Kontext:
{% for doc in documents %}
{{ doc.content }} Partei:{{ doc.meta['partei'] }} Quellenname:{{ doc.meta['name'] }}
{% endfor %}

Anweisung:
Beantworte die Frage des Nutzers basierend auf dem gegebenen Kontext, mache dabei klar, welche Partei welche Position vertritt. Vermeide Wiederholungen und gebe nur Antworten auf die gestellte Frage.
Du kannst beispielweise einen Absatz pro Partei schreiben, oder ähnliche Positionen zusammenfassen.
Verwende nur Informationen aus dem gegebenen Kontext und füge nichts hinzu.
Wenn eine Partei nicht erwähnt wird, lasse sie aus der Antwort weg.
Wenn der Kontext insgesamt nicht genug Informationen enthält, gib das am Ende der Antwort an.

Frage: {{query}}"""

# https://huggingface.co/TheBloke/SauerkrautLM-Mixtral-8x7B-GGUF#prompt-template-chatml
m8x7b_template = """<|im_start|>system
Beantworte die Frage des Nutzers basierend auf dem gegebenen Kontext, mache dabei klar, welche Partei welche Position vertritt. Vermeide Wiederholungen und gebe nur Antworten auf die gestellte Frage.
Du kannst beispielweise einen Absatz pro Partei schreiben, oder ähnliche Positionen zusammenfassen.
Verwende nur Informationen aus dem gegebenen Kontext und füge nichts hinzu.
Wenn eine Partei nicht erwähnt wird, lasse sie aus der Antwort weg.
Wenn der Kontext insgesamt nicht genug Informationen enthält, gib das am Ende der Antwort an.<|im_end|>
<|im_start|>user
Kontext:
{% for doc in documents %}
{{ doc.content }} Partei:{{ doc.meta['partei'] }} Quellenname:{{ doc.meta['name'] }}
{% endfor %}

Frage: {{query}}<|im_end|>
<|im_start|>assistant"""

pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(
    model="T-Systems-onsite/cross-en-de-roberta-sentence-transformer",
    prefix="Berechne Embeddings für die folgenden deutschen Sätze. Die Embeddings sollen die Semantik der Sätze gut repräsentieren, sodass ähnliche Sätze ähnliche Vektoren ergeben: "
))
pipe.add_component("retriever", ChromaEmbeddingRetriever(document_store=document_store))
# pipe.add_component("prompt_builder", PromptBuilder(template=mistral7b_template))
pipe.add_component("prompt_builder", PromptBuilder(template=m8x7b_template))
pipe.add_component("llm", generator)

pipe.connect("embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")


# define interaction functions
def get_answer(message: str) -> str:
    question = (
        message
    )

    result = pipe.run(
        {
            "embedder": {"text": question},
            "prompt_builder": {"query": question}
        }
    )

    return result["llm"]["replies"][0]

## Evaluation

Im Folgenden wird der Chatbot mit verschiedenen Fragen aus verschiedenen Kategorien getestet. Die Antworten werden bewertet und die Stärken und Schwächen des Chatbots werden am Ende des Notebooks zusammengefasst.

In [11]:
import re


## Setup utility functions

def ask_question(message: str):
    print(f"User: {message}")
    text = get_answer(message)

    print(f"Assistant: {text}")
    print()
    print()

### Klimawandel

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Umwelt- und Klimafragen.

In [12]:
ask_question(
    "Ich bin besorgt über den Klimawandel. Welche Parteien haben konkrete Maßnahmen zur Bekämpfung des Klimawandels in ihrem Wahlprogramm?"
)

ask_question(
    "Welche Parteien setzen sich für den Klimaschutz und betreiben ansonsten eine liberale Wirtschaftspolitik?"
)

ask_question(
    "Welche Parteien setzen sich für den Klimaschutz ein und haben eine konservative Wirtschaftspolitik?"
)

ask_question(
    "Gibt es politisch rechte Parteien, die den Klimawandel leugnen?"
)

ask_question(
    "Gibt es Parteien, die eine Abkehr von der Atomkraft in Europa fordern?"
)

llama_model_loader: loaded meta data with 24 key-value pairs and 995 tensors from ./.cache/models--TheBloke--SauerkrautLM-Mixtral-8x7B-GGUF/snapshots/835db1d04dee093a982e7fb397f6f2ddb1edf6f5/sauerkrautlm-mixtral-8x7b.Q3_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = vagosolutions_sauerkrautlm-mixtral-8x7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6

User: Ich bin besorgt über den Klimawandel. Welche Parteien haben konkrete Maßnahmen zur Bekämpfung des Klimawandels in ihrem Wahlprogramm?


....................................................................................................
llama_new_context_with_model: n_ctx      = 16384
llama_new_context_with_model: n_batch    = 128
llama_new_context_with_model: n_ubatch   = 128
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 1000000.0
llama_new_context_with_model: freq_scale = 1
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M3 Pro
ggml_metal_init: picking default device: Apple M3 Pro
ggml_metal_init: using embedded metal library
ggml_metal_init: GPU name:   Apple M3 Pro
ggml_metal_init: GPU family: MTLGPUFamilyApple9  (1009)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction support   = true
ggml_metal_init: simdgroup matrix mul. support = true
ggml_metal_init: hasUnifiedMemory              = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 28991.03 MB
llama_kv

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      40.51 ms /   549 runs   (    0.07 ms per token, 13551.87 tokens per second)
llama_print_timings: prompt eval time =   70151.43 ms /  5503 tokens (   12.75 ms per token,    78.44 tokens per second)
llama_print_timings:        eval time =   54962.10 ms /   548 runs   (  100.30 ms per token,     9.97 tokens per second)
llama_print_timings:       total time =  125379.76 ms /  6051 tokens


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      15.72 ms /   215 runs   (    0.07 ms per token, 13678.59 tokens per second)
llama_print_timings: prompt eval time =   52310.68 ms /  5256 tokens (    9.95 ms per token,   100.48 tokens per second)
llama_print_timings:        eval time =   21219.59 ms /   214 runs   (   99.16 ms per token,    10.09 tokens per second)
llama_print_timings:       total time =   73599.09 ms /  5470 tokens


Assistant: 
Ja, politisch rechte Parteien, die den Klimawandel leugnen, gibt es. Ein Beispiel ist die deutsche AfD (Alternative für Deutschland), die in ihrem Europawahlprogramm 2024 die Existenz des durch Menschen verursachten Klimawandels bestreitet und eine EU-weite Regulierung und Vorschreibung von Maßnahmen gegen den Klimawandel ablehnt. Die AfD betont in ihrem Programm auch die Bedeutung der Energievielfalt und lehnt das Erreichen von "Netto- Null-CO2-Emissionen" bis 2040 ab. Es ist jedoch wichtig zu beachten, dass nicht alle rechten Parteien den Klimawandel leugnen, und die Haltungen zu Umwelt- und Energiefragen können von einer rechten Partei zur anderen unterschiedlich sein.


User: Gibt es Parteien, die eine Abkehr von der Atomkraft in Europa fordern?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      12.67 ms /   170 runs   (    0.07 ms per token, 13414.35 tokens per second)
llama_print_timings: prompt eval time =   53957.12 ms /  5419 tokens (    9.96 ms per token,   100.43 tokens per second)
llama_print_timings:        eval time =   16846.53 ms /   169 runs   (   99.68 ms per token,    10.03 tokens per second)
llama_print_timings:       total time =   70861.06 ms /  5588 tokens


Assistant: 
Ja, die Volt-Partei fordert eine Abkehr von der Atomkraft in Europa. Im Europäischen Wahlprogramm 2024–2029 von Volt Europa wird betont, dass der Übergang zu einer kohlenstofffreien Wirtschaft und die Bekämpfung des Klimawandels wichtige Prioritäten sind. Dabei sollen erneuerbare Energien gefördert werden, während auf die Nutzung von Atomkraft verzichtet wird. Es wird betont, dass Investitionen in die Atomenergie nicht mit den Zielen für eine kohlenstofffreie Wirtschaft und eine nachhaltige Umwelt vereinbar sind.




### Evaluation Klimawandel

1. Gut strukturierte und kompakte Antworten auf die Frage. Auch sehr gut gegliedert nach Parteien. → gute Antwort
2. Fokus auf Die Linke und Grüne, obwohl diese keine klassisch liberale Wirtschaftspolitik verfolgen. Jedoch haben ihre Wirtschaftspolitiken auch liberale Elemente. → ausreichend gute Antwort
3. Ähnlich wie bei 2. uneingeschränkte zuordnung CDU/CSU und FDP zur konservativen Wirtschaftspolitik, trotzdem eine informierte Antwort. → ausreichend gute Antwort
4. Korrekte Zuordnung der AfD, mit ausführlicher Erklärung. → gute Antwort
5. Fokus auf die Volt Partei, ignoriert jedoch die Grünen und Die Linke, die ebenfalls eine Abkehr von der Atomkraft fordern. → mangelhafte Antwort

### Verteidigung und Außenpolitik

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Verteidigungs- und Außenpolitik.

In [13]:
ask_question(
    "Welche Parteien fordern in ihrem Wahlprogramm eine Stärkung der europäischen Verteidigungspolitik?"
)

ask_question(
    "Welche Parteien sprechen sich klar gegen eine weitere EU-Erweiterung aus?"
)

ask_question(
    "Welche Parteien setzen sich für eine engere Zusammenarbeit mit dem Vereinigten Königreich nach dem Brexit ein?"
)

User: Welche Parteien fordern in ihrem Wahlprogramm eine Stärkung der europäischen Verteidigungspolitik?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      19.84 ms /   268 runs   (    0.07 ms per token, 13507.38 tokens per second)
llama_print_timings: prompt eval time =   53948.32 ms /  5406 tokens (    9.98 ms per token,   100.21 tokens per second)
llama_print_timings:        eval time =   26635.33 ms /   267 runs   (   99.76 ms per token,    10.02 tokens per second)
llama_print_timings:       total time =   80675.22 ms /  5673 tokens


Assistant: 
In ihren Wahlprogrammen fordern die Sozialdemokratische Partei Deutschlands (SPD), Bündnis 90/Die Grünen, Die Freie Wähler und Bündnis Deutschland eine Stärkung der europäischen Verteidigungspolitik. Die SPD möchte Europa in die Lage versetzen, Sicherheit vor externen Bedrohungen zunehmend eigenständig zu gewährleisten und eine nachhaltige Verteidigungsfinanzierung von mindestens 2 Prozent des Bruttoinlandsproduktes einzuführen. Die Grünen fordern eine gemeinsame europäische Armee, die unter demokratischer Kontrolle steht und in Zusammenarbeit mit der Bundeswehr aufgebaut wird. Die Freie Wähler möchten eine stärkere Integration der EU-Mitgliedsstaaten im Bereich Verteidigungspolitik, um gemeinsame Einsätze und Missionen effizienter gestalten zu können. Bündnis Deutschland betont die Notwendigkeit einer europäischen Landesverteidigung, die auch ohne Unterstützung der USA gewährleistet ist.


User: Welche Parteien sprechen sich klar gegen eine weitere EU-Erweiterung aus?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      10.22 ms /   139 runs   (    0.07 ms per token, 13606.11 tokens per second)
llama_print_timings: prompt eval time =   55155.16 ms /  5513 tokens (   10.00 ms per token,    99.95 tokens per second)
llama_print_timings:        eval time =   13796.95 ms /   138 runs   (   99.98 ms per token,    10.00 tokens per second)
llama_print_timings:       total time =   68999.66 ms /  5651 tokens


Assistant: 
Die AfD (Alternative für Deutschland) spricht sich klar gegen eine weitere EU-Erweiterung aus. Im gegebenen Kontext wird die EU-Erweiterung von der CDU, FDP, Bündnis 90/Die Grünen und Volt Europa unterstützt, während die AfD eine EU-Erweiterung als "nicht verantwortbar" und "gefährlich" für die EU und ihre Mitgliedstaaten ansieht. Die übrigen erwähnten Parteien haben keine klare Stellung gegen eine weitere EU-Erweiterung eingenommen.


User: Welche Parteien setzen sich für eine engere Zusammenarbeit mit dem Vereinigten Königreich nach dem Brexit ein?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      11.47 ms /   154 runs   (    0.07 ms per token, 13428.67 tokens per second)
llama_print_timings: prompt eval time =   52593.61 ms /  5270 tokens (    9.98 ms per token,   100.20 tokens per second)
llama_print_timings:        eval time =   15176.45 ms /   153 runs   (   99.19 ms per token,    10.08 tokens per second)
llama_print_timings:       total time =   67822.59 ms /  5423 tokens


Assistant: 
Die CDU/CSU, die FDP und die Grünen setzen sich für eine engere Zusammenarbeit mit dem Vereinigten Königreich nach dem Brexit ein. Die CDU/CSU möchten im großen gegenseitigen strategischen Interesse zusammenarbeiten, auch in der Sicherheitspolitik. Die FDP steht dafür ein, eine flexible Integration zu ermöglichen und die Zusammenarbeit mit Großbritannien zu vertiefen. Die Grünen möchten eine neue Partnerschaft zwischen der EU und dem Vereinigten Königreich fördern und dabei auch auf eine enge wirtschaftliche und militärische Zusammenarbeit achten.




### Evaluation Verteidigung und Außenpolitik

1. Die Ansichten der erwähnten Parteien sind klar und prägnant dargestellt. Die Union wird leider übergangen, obwohl diese auch eine Stärkung der europäischen Verteidigungspolitik fordert. → ausreichend gute Antwort
2. Die Antwort ist sehr hilfreich, das sie nicht nur die AfD beschreibt, sondern auch Gegenbeispiele nennt. → sehr gute Antwort
3. Die Antwort ist hilfreich, da sie die Überlappungen der Ansichten der Parteien aufzeigt, aber auch die Unterschiede bei der Umsetzung aufzeigt. → sehr gute Antwort

### Wirtschafts- und Finanzpolitik

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Wirtschafts- und Finanzpolitik.

In [14]:
ask_question(
    "Gibt es Parteien, die sich für eine Lockerung der Schuldenregeln in der EU einsetzen?"
)

ask_question(
    "Welche Parteien setzen sich für eine stärkere Regulierung von Big-Tech-Unternehmen ein?"
)

ask_question(
    "Welche Parteien setzen sich für eine stärkere Besteuerung von multinationalen Konzernen ein?"
)

User: Gibt es Parteien, die sich für eine Lockerung der Schuldenregeln in der EU einsetzen?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      16.29 ms /   223 runs   (    0.07 ms per token, 13686.02 tokens per second)
llama_print_timings: prompt eval time =   54204.41 ms /  5436 tokens (    9.97 ms per token,   100.29 tokens per second)
llama_print_timings:        eval time =   22189.74 ms /   222 runs   (   99.95 ms per token,    10.00 tokens per second)
llama_print_timings:       total time =   76469.14 ms /  5658 tokens


Assistant: 
Ja, die Linke und Bündnis 90/Die Grünen setzen sich für eine Lockerung der Schuldenregeln in der EU ein. Die Linke fordert eine grundlegende Reform der Finanzen der Europäischen Union durch eine Verdreifachung des EU-Haushalts, um öffentliche Investitionen zu ermöglichen und soziale Ungleichgewichte zu beseitigen. Bündnis 90/Die Grünen möchten die finanzielle Ausstattung der EU insgesamt durch neue Eigenmittel und höhere nationale Beiträge verbessern und für einen starken Mehrjährigen Finanzrahmen (MFR) streiten, um gemeinsame Investitionen in strategisch wichtigen Bereichen zu ermöglichen. Beide Parteien lehnen eine Vergrößerung des EU-Personalbestands ab und möchten die EU weiterentwickeln.


User: Welche Parteien setzen sich für eine stärkere Regulierung von Big-Tech-Unternehmen ein?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      10.74 ms /   146 runs   (    0.07 ms per token, 13599.11 tokens per second)
llama_print_timings: prompt eval time =   55116.35 ms /  5530 tokens (    9.97 ms per token,   100.33 tokens per second)
llama_print_timings:        eval time =   14515.41 ms /   145 runs   (  100.11 ms per token,     9.99 tokens per second)
llama_print_timings:       total time =   69677.50 ms /  5675 tokens


Assistant: 
Die Grünen und die Linke setzen sich für eine stärkere Regulierung von Big-Tech-Unternehmen ein. Die Grünen fordern eine konsequente Durchsetzung der Datenschutz-Grundverordnung (DSGVO) und eine Evaluation der DSGVO, um die Regelungen zu vereinfachen und für KMU und Start-ups anzupassen. Die Linke möchten Maßnahmen gegen Monopolisierung und Datenmissbrauch ergreifen, um eine stärkere Regulierung und mehr Transparenz bei Big-Tech-Unternehmen zu erreichen.


User: Welche Parteien setzen sich für eine stärkere Besteuerung von multinationalen Konzernen ein?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      32.74 ms /   451 runs   (    0.07 ms per token, 13776.46 tokens per second)
llama_print_timings: prompt eval time =   52408.56 ms /  5265 tokens (    9.95 ms per token,   100.46 tokens per second)
llama_print_timings:        eval time =   44865.38 ms /   450 runs   (   99.70 ms per token,    10.03 tokens per second)
llama_print_timings:       total time =   97446.39 ms /  5715 tokens


Assistant: 
Die AfD lehnt eine stärkere Besteuerung von multinationalen Konzernen ab und ist der Auffassung, dass es im 21. Jahrhundert nicht mehr zeitgemäß ist, Großkonzerne danach zu besteuern, wo sie ihren Unternehmenssitz haben. Maßgeblich für die Steuerlast sollte die eigentliche wirtschaftliche Aktivität sein, die ein Konzern in einem Land entfaltet.

Die Grünen und Die Linke setzen sich hingegen für eine stärkere Besteuerung von multinationalen Konzernen ein. Die Grünen unterstützen die Mitgliedstaaten bei der Einführung und Umsetzung von geeigneten Maßnahmen zur Reduzierung der Vermögensungleichheit und wollen verhindern, dass der Bäckerladen um die Ecke einen deutlich höheren Steuersatz zahlt als ein internationaler Großkonzern. Die Linke fordert eine EU-Mindestbesteuerung von 15 Prozent auf OECD-Ebene und eine einheitliche Basis-Körperschaftssteuer von 15 Prozent auf einer einheitlichen Bemessungsgrundlage in Europa.

Die SPD fordert im Rahmen ihrer EU-Taxonomie eine zentral 

### Evaluation Wirtschafts- und Finanzpolitik

1. Die Antwort ist sehr hilfreich, da sie einen guten Überblick über die relevanten Parteien und ihre Positionen gibt. → sehr gute Antwort
2. Analog zu 1. mit guter Einordnung und Gegenüberstellung → sehr gute Antwort
3. Die Antwort bietet eine exzellente Einordnung der Parteien und ihrer Positionen, wobei ähnliche Positionen gruppiert erläutert werden. → sehr gute Antwort

### Institutionelle Reformen und Entscheidungsprozesse

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu institutionellen Reformen und Entscheidungsprozessen in der EU.

In [15]:
ask_question(
    "Gibt es Parteien, die eine Ausweitung des Mehrheitsprinzips anstelle von Einstimmigkeit bei EU-Entscheidungen fordern?"
)

User: Gibt es Parteien, die eine Ausweitung des Mehrheitsprinzips anstelle von Einstimmigkeit bei EU-Entscheidungen fordern?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      15.12 ms /   206 runs   (    0.07 ms per token, 13627.94 tokens per second)
llama_print_timings: prompt eval time =   53339.22 ms /  5378 tokens (    9.92 ms per token,   100.83 tokens per second)
llama_print_timings:        eval time =   20431.18 ms /   205 runs   (   99.66 ms per token,    10.03 tokens per second)
llama_print_timings:       total time =   73839.38 ms /  5583 tokens


Assistant: 
Ja, es gibt mehrere Parteien, die eine Ausweitung des Mehrheitsprinzips anstelle von Einstimmigkeit bei EU-Entscheidungen fordern. Im gegebenen Kontext sind das beispielsweise die CDU/CSU, die Grünen und die FDP. Diese Parteien möchten durch Anwendung der Passerelle-Klausel Entscheidungen schneller ermöglichen und eine stärkere Nutzung der in den Verträgen vorgesehenen Klausel der Verstärkten Zusammenarbeit. Damit eine erweiterte EU handlungsfähig bleibt, muss sie ihre Strukturen reformieren, so die Forderung. Erweiterung und Reformen müssen Hand in Hand gehen, um eine effektive gemeinsame Entscheidungsfindung zu gewährleisten.




### Evaluation institutionelle Reformen und Entscheidungsprozesse

1. Die Antwort ist sehr informativ, deckt aber nicht alle Parteien ab, die sich für eine Ausweitung des Mehrheitsprinzips aussprechen. → ausreichend gute Antwort

### Migrations- und Asylpolitik

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Migrations- und Asylpolitik.

In [16]:
ask_question(
    "Fordern einige Parteien eine Reform des EU-Asylsystems? Wenn ja, welche konkreten Vorschläge machen sie?"
)

ask_question(
    "Gibt es Parteien, die sich für eine stärkere Kontrolle der EU-Außengrenzen mit Hilfe von KI-Technologien einsetzen?"
)

User: Fordern einige Parteien eine Reform des EU-Asylsystems? Wenn ja, welche konkreten Vorschläge machen sie?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      33.97 ms /   395 runs   (    0.09 ms per token, 11629.62 tokens per second)
llama_print_timings: prompt eval time =   52988.02 ms /  5329 tokens (    9.94 ms per token,   100.57 tokens per second)
llama_print_timings:        eval time =   40413.12 ms /   394 runs   (  102.57 ms per token,     9.75 tokens per second)
llama_print_timings:       total time =   93629.49 ms /  5723 tokens


Assistant: 
Ja, einige Parteien fordern eine Reform des EU-Asylsystems. Im gegebenen Kontext sind das die PARTEI MENSCH UMWELT TIERSCHUTZ, Bündnis Deutschland, SPD und FDP.

Die PARTEI MENSCH UMWELT TIERSCHUTZ fordert verpflichtende Aufenthaltszonen an den EU-Grenzen sowie außerhalb der EU unter EU-Kontrolle und humanes Entscheidungsverfahren für die Aufnahme von Geflüchteten.

Bündnis Deutschland möchte auch verpflichtende Aufenthaltszonen, aber mit EU-Kontrolle innerhalb der EU, und eine einmalige Entscheidung über das Asylgesuch direkt bei Erreichen der ersten EU-Außengrenze.

Die SPD fordert sichere und legale Fluchtwege, legale Zugangswege für geflüchtete Menschen und humanitäre Visa. Sie möchten auch eine EU-Strategie zur Bekämpfung von Fluchtursachen.

Die FDP möchte die Integration von zugewanderten Fachkräften und die Eingliederung von sonstigen Migranten in Arbeitsmarkt und Gesellschaft fördern, um Parallelgesellschaften zu verhindern.

Die Volt-Partei schlägt ein effizientes

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      56.39 ms /   750 runs   (    0.08 ms per token, 13300.47 tokens per second)
llama_print_timings: prompt eval time =   56040.49 ms /  5511 tokens (   10.17 ms per token,    98.34 tokens per second)
llama_print_timings:        eval time =   76189.92 ms /   749 runs   (  101.72 ms per token,     9.83 tokens per second)
llama_print_timings:       total time =  132637.79 ms /  6260 tokens


Assistant: 
Ja, die FDP (Freie Demokraten) und Bündnis Deutschland sind zwei solche Parteien. Beide unterstützen eine stärkere Sicherung der EU-Außengrenzen mit Hilfe von KI-Technologien. Die FDP betont in ihrem Europawahlprogramm die Notwendigkeit, dass Europa zu -
sammenschrumpft und die EU-Beitrittsperspektive als zentrales Instrument zur Sicherung von Demokratie, innerer und äußerer Sicherheit, politischer Stabilität und wirtschaftlichen Wohlstand nutzt. Bündnis Deutschland spricht sich in seinem Europawahlprogramm für eine verstärkte Nutzung von KI-Technologien aus, um die Sicherheit der EU nach innen und außen zu stärken. Beide Parteien betonen dabei die Notwendigkeit, dass Grenzkontrollen und Schlagbäume innerhalb Europas vermieden werden müssen. 

Die FDP möchte eine EU-Beitrittsperspektive für alle EU-Bewerberländer schaffen und dabei den Fokus auf Rechtsstaatlichkeit legen. Die Türkei, die von Präsident Erdogan autoritär regiert wird, kann nach Ansicht der FDP kein Kandidat f

### Evaluation Migrations- und Asylpolitik

1. Die Antwort fasst viele Positionen treffend zusammen, jedoch fehlen einige Parteien, die ebenfalls eine Reform des EU-Asylsystems fordern. Die Tierschutzpartei wird unter ihrem vollen Namen (PARTEI MENSCH UMWELT TIERSCHUTZ), was einige Nutzer verwirren könnte. → gute Antwort
2. Die Antwort ist sehr ausführlich und schweift dabei von der eigentlichen Fragestellung ab. Viele Positionen werden ausführlich dargestellt, wodurch das Tokenlimit erreicht wird, was zu einer unvollständigen Antwort führt. → akzeptable

### Verbraucherschutz 

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Verbraucherschutzthemen.

In [17]:
ask_question(
    "Welche Parteien sprechen sich für eine Stärkung des EU-Verbraucherschutzes aus?"
)

ask_question(
    "Welche Parteien setzen sich für eine stärkere Regulierung von Lebensmittelzusatzstoffen in der EU ein?"
)

ask_question(
    "Gibt es Parteien, die sich für eine stärkere Regulierung von Tabakwerbung in der EU einsetzen?"
)

User: Welche Parteien sprechen sich für eine Stärkung des EU-Verbraucherschutzes aus?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      17.33 ms /   230 runs   (    0.08 ms per token, 13274.85 tokens per second)
llama_print_timings: prompt eval time =   53439.54 ms /  5367 tokens (    9.96 ms per token,   100.43 tokens per second)
llama_print_timings:        eval time =   22981.65 ms /   229 runs   (  100.36 ms per token,     9.96 tokens per second)
llama_print_timings:       total time =   76514.53 ms /  5596 tokens


Assistant: 
Die Parteien, die sich im gegebenen Kontext für eine Stärkung des EU-Verbraucherschutzes aussprechen, sind Bündnis 90/Die Grünen und Die Linke. Bündnis 90/Die Grünen fordern unter anderem die Einführung von Resilienz- und Nachhaltigkeitskriterien bei öffentlichen Ausschreibungen und öffentlichen Förderprogrammen, um gezielt die Produktion dieser Produkte in der EU zu begünstigen. Die Linke möchten den Europäischen Betriebsräten eine stärkere Rolle zukommen lassen, indem sie die EU-Kommission auffordern, die bestehende Richtlinie zu überarbeiten und Ausnahmeregeln zu beenden. Darüber hinaus wollen sie den Schutz der öffentlichen Daseinsvorsorge stärken und das Privatisierungsverbot im Bereich Wasser weiter ausbauen.


User: Welche Parteien setzen sich für eine stärkere Regulierung von Lebensmittelzusatzstoffen in der EU ein?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      13.19 ms /   177 runs   (    0.07 ms per token, 13415.19 tokens per second)
llama_print_timings: prompt eval time =   54653.78 ms /  5455 tokens (   10.02 ms per token,    99.81 tokens per second)
llama_print_timings:        eval time =   17615.24 ms /   176 runs   (  100.09 ms per token,     9.99 tokens per second)
llama_print_timings:       total time =   72341.55 ms /  5631 tokens


Assistant: 
Die FREIE WÄHLER und die TIERSCHUTZPARTEI setzen sich für eine stärkere Regulierung von Lebensmittelzusatzstoffen in der EU ein. Die FREIE WÄHLER fordern, dass bestehende Gesetze konsequent durchgesetzt werden und bei Nichteinhaltung sanktioniert werden. Die TIERSCHUTZPARTEI möchte eine bio-vegane Landwirtschaft fördern, die gesundheitsschädlicher Stoffe in unseren Lebensmitteln reduzieren kann. Beide Parteien betonen die Bedeutung von Gesetzen und Regulierungen für den Schutz der Verbraucher*innen und die Sicherstellung einer sauberen Umwelt.


User: Gibt es Parteien, die sich für eine stärkere Regulierung von Tabakwerbung in der EU einsetzen?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      21.72 ms /   286 runs   (    0.08 ms per token, 13168.19 tokens per second)
llama_print_timings: prompt eval time =   54380.61 ms /  5296 tokens (   10.27 ms per token,    97.39 tokens per second)
llama_print_timings:        eval time =   28431.65 ms /   285 runs   (   99.76 ms per token,    10.02 tokens per second)
llama_print_timings:       total time =   82935.98 ms /  5581 tokens


Assistant: 
Ja, die Linke und Bündnis 90/Die Grünen sind zwei politische Parteien, die sich für eine stärkere Regulierung von Tabakwerbung in der EU einsetzen. Im Kontext der gegebenen Informationen wird deutlich, dass beide Parteien Maßnahmen zur Eindämmung von Tabakprodukten und deren Werbung fordern. Die Linke möchte die Direktvergabe der Daseinsvorsorge an kommunale Träger stärken und das Privatisierungsverbot im Bereich Wasser ausbauen, während Bündnis 90/Die Grünen eine bessere finanzielle Ausstattung der EU fordern und sich für eine nachhaltige Industriepolitik einsetzen. Beide Parteien legen Wert auf die Sicherstellung von Arzneimittelversorgung und stellen gemeinwohl-orientierte Prinzipien über den freien Binnenmarkt. Es ist anzunehmen, dass sie auch eine stärkere Regulierung von Tabakwerbung unterstützen würden, um die öffentliche Gesundheit zu schützen und das Rauchverhalten in der EU einzudämmen.




### Evaluation Verbraucherschutz

1. Die Antwort ist sehr informativ, geht jedoch nicht auf die Rechte ver Vebrauchern ein, sondern auf die Regulierung von Herstellungsprozessen. → akzeptable Antwort
2. Die Antwort ist informativ und gibt einen interessanten Überblick über zwei ähnliche Positionen, die siech jedoch in der Umsetzung unterscheiden. → gute Antwort
3. Die Antwort ist vor allem am Ende aufgrund der guten Einordnung der Parteien informativ, zwischendurch wird bei der Antwort jedoch von der eigentlichen Fragestellung abgewichen. → akzeptable Antwort

### Agrarpolitik

Die folgenden Fragen beziehen sich auf die Positionen der Parteien zu Agrarpolitik und Landwirtschaft.

In [18]:
ask_question(
    "Fordern einige Parteien eine Reform der Gemeinsamen Agrarpolitik der EU? Wenn ja, in welche Richtung soll diese gehen?"
)

ask_question(
    "Welche Parteien setzen sich für eine stärkere Förderung von ökologischer Landwirtschaft in der EU ein?"
)

User: Fordern einige Parteien eine Reform der Gemeinsamen Agrarpolitik der EU? Wenn ja, in welche Richtung soll diese gehen?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      24.33 ms /   329 runs   (    0.07 ms per token, 13522.96 tokens per second)
llama_print_timings: prompt eval time =   53326.53 ms /  5376 tokens (    9.92 ms per token,   100.81 tokens per second)
llama_print_timings:        eval time =   32760.84 ms /   328 runs   (   99.88 ms per token,    10.01 tokens per second)
llama_print_timings:       total time =   86221.13 ms /  5704 tokens


Assistant: 
Ja, einige Parteien fordern eine Reform der Gemeinsamen Agrarpolitik (GAP) der EU. Die Grünen und die Freie Demokraten (FDP) sind dafür, dass die GAP stärker auf Umwelt- und Klimaschutz ausgerichtet wird.

Die Grünen möchten eine Vergabe von EU-Geldern an die Einrichtung von Ausbildungsplätzen, das Einhalten von Sozialstandards und das Befolgen geltender Tarife koppeln. Damit sollen Arbeitsplätze in der Industrie erhalten und gleichzeitig die Umwelt geschont werden.

Die FDP möchte eine vielfältige und vereinte diplomatische Vertretung mit europäischer Stimme schaffen, um gemeinsam eine nachhaltige Agrarpolitik zu gestalten. Dabei sollen auch unterschiedliche historische Kontexte in Europa berücksichtigt werden.

Die Forderungen der anderen im gegebenen Kontext genannten Parteien (Volt, Bündnis 90/Die Grünen, FDP, CSU und AfD) beziehen sich hauptsächlich auf die Finanzierung der EU, die Erweiterung der EU oder die Stärkung der Wirtschaft und Sozialsysteme. Eine Reform der G

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      22.44 ms /   298 runs   (    0.08 ms per token, 13279.27 tokens per second)
llama_print_timings: prompt eval time =   55468.07 ms /  5461 tokens (   10.16 ms per token,    98.45 tokens per second)
llama_print_timings:        eval time =   29822.09 ms /   297 runs   (  100.41 ms per token,     9.96 tokens per second)
llama_print_timings:       total time =   85418.31 ms /  5758 tokens


Assistant: 
Bündnis Deutschland, die SPD, Bündnis 90/Die Grünen und Volt Europa setzen sich für eine stärkere Förderung von ökologischer Landwirtschaft in der EU ein. Bündnis Deutschland fordert, die Aufnahme weiterer landwirtschaftlicher Betriebe in die Richtlinie zu Industrieemissionen zu stoppen. Die SPD möchte ein Gesetz zur Bodengesundheit verabschieden und ausreichende Mittel für die Wiederherstellung der Natur bereitstellen. Bündnis 90/Die Grünen wollen eine zeitlich gestaffelte Einführung von Resilienz- und Nachhaltigkeitskriterien bei öffentlichen Ausschreibungen und Förderprogrammen umsetzen, um die Produktion von klimaneutralen Produkten zu begünstigen. Volt Europa schlägt eine Verdreifachung des Budgets für Forschungs- und Innovationsprogramme vor, um Projekte zur Bewältigung der Klimakrise zu unterstützen. 

Die übrigen erwähnten Parteien (FDP und AfD) haben im gegebenen Kontext keine Hinweise auf eine stärkere Förderung von ökologischer Landwirtschaft geliefert.




### Evaluation Agrarpolitik

1. Die Antwort ist sehr eine sehr gute Auswahl an bestehenden explizit formulierten Positionen, die auch erwähnt welche Parteien keine expliziten Forderungen haben. → sehr gute Antwort
2. Die Antwort ist sehr informativ und liefert einen guten Einblick. Als Wähler hätte ich Positionen von der Tierschutzpartei und vielleicht noch einigen anderen erwartet. → gute Antwort

### Persönliche Fragestellungen

Die folgenden Fragen beziehen sich auf persönliche Anliegen der Nutzer und dienen dazu herauszufinden, wie gut der Chatbot auf individuelle Wünsche eingehen kann.

In [19]:
ask_question(
    "Ich bin Anfang 20 und mache mir Sorgen um meine Rente in der Zukunft. Ich habe kein Vertrauen in effektive Maßnahmen der Grünen, bin aber trotzdem sehr besorgt wegen des Klimawandels. Welche Parteien sind gute Optionen für mich?"
)

ask_question(
    "Ich bin Landwirt und befürchte, dass eine zu strikte Klimapolitik meine Existenz bedroht. Welche Parteien setzen sich für die Interessen der Landwirte ein und haben trotzdem Klimaschutz im Blick?"
)

ask_question(
    "Als Unternehmer wünsche ich mir weniger Bürokratie in der EU. Welche Parteien setzen sich für den Bürokratieabbau ein, ohne dabei den Verbraucherschutz zu vernachlässigen?"
)

ask_question(
    "Ich lebe in einer Grenzregion und profitiere vom Schengen-Abkommen. Welche Parteien setzen sich klar für offene Grenzen innerhalb Europas ein?"
)

User: Ich bin Anfang 20 und mache mir Sorgen um meine Rente in der Zukunft. Ich habe kein Vertrauen in effektive Maßnahmen der Grünen, bin aber trotzdem sehr besorgt wegen des Klimawandels. Welche Parteien sind gute Optionen für mich?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      34.31 ms /   468 runs   (    0.07 ms per token, 13638.75 tokens per second)
llama_print_timings: prompt eval time =   52100.06 ms /  5185 tokens (   10.05 ms per token,    99.52 tokens per second)
llama_print_timings:        eval time =   46392.91 ms /   467 runs   (   99.34 ms per token,    10.07 tokens per second)
llama_print_timings:       total time =   98686.84 ms /  5652 tokens


Assistant: 
Die Linke und die Piratenpartei vertreten eine ähnliche Position hinsichtlich einer europaweiten Mindestrente, die sicherstellt, dass alle Menschen im Alter vor Armut geschützt sind. Beide Parteien setzen sich auch für eine Verbesserung der Rentensysteme und gegen das Spekulieren mit Rentenleistungen ein.

Die Sozialdemokratische Partei (SPD) legt in ihrem Programm einen Fokus auf den sozialen Klima- und Umweltschutz, um eine gerechte Energiewende zu gewährleisten. Die SPD möchte sicherstellen, dass alle Bürgerinnen und Bürger den Weg bis 2050 zur Klimaneutralität gemeinsam und souverän gestalten können.

Die Bündnis 90/Die Grünen stellen die ökologische Transformation in den Mittelpunkt ihrer politischen Arbeit und wollen eine gerechte Energiewende, die alle Menschen miteinbezieht. Sie setzen sich für eine Arbeitszeitverkürzung auf 30 Wochenstunden ein, um eine bessere Work-Life-Balance zu gewährleisten.

Die Tierschutzpartei legt in ihrem Programm einen besonderen Fokus a

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      14.09 ms /   191 runs   (    0.07 ms per token, 13554.75 tokens per second)
llama_print_timings: prompt eval time =   54489.59 ms /  5376 tokens (   10.14 ms per token,    98.66 tokens per second)
llama_print_timings:        eval time =   19081.79 ms /   191 runs   (   99.90 ms per token,    10.01 tokens per second)
llama_print_timings:       total time =   73647.60 ms /  5567 tokens


Assistant: 
Die FDP und die AfD setzen sich im gegebenen Kontext für den Bürokratieabbau in der EU ein. Die FDP möchte eine Überprüfung und Reduzierung von EU-Richtlinien, um bürokratische Hürden abzubauen und dabei die Einhaltung von Sozial-, Umwelt- und Sicherheitsstandards zu gewährleisten. Die AfD fordert eine Überprüfung und Reduzierung der einschlägigen EU-Richtlinien, um Bürokratie abzubauen und eine Wettbewerbsgleichheit für Unternehmen zu schaffen. Beide Parteien betonen jedoch auch die Notwendigkeit, den Verbraucherschutz und faire Wettbewerbsbedingungen zu gewährleisten.


User: Ich lebe in einer Grenzregion und profitiere vom Schengen-Abkommen. Welche Parteien setzen sich klar für offene Grenzen innerhalb Europas ein?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   16399.01 ms
llama_print_timings:      sample time =      15.84 ms /   215 runs   (    0.07 ms per token, 13572.38 tokens per second)
llama_print_timings: prompt eval time =   53181.92 ms /  5302 tokens (   10.03 ms per token,    99.70 tokens per second)
llama_print_timings:        eval time =   21295.31 ms /   214 runs   (   99.51 ms per token,    10.05 tokens per second)
llama_print_timings:       total time =   74559.65 ms /  5516 tokens


Assistant: 
Die FDP (Freie Demokraten), Bündnis 90/Die Grünen und die SPD (Sozialdemokratische Partei Deutschlands) setzen sich klar für offene Grenzen innerhalb Europas ein. Die FDP betont die Freiheit des Einzelnen und die Notwendigkeit einer vielfältigen, selbstbestimmten und offenen Gesellschaft. Bündnis 90/Die Grünen stehen für eine EU, die verbindliche Standards setzt, um faire Löhne und starke Gewerkschaften zu gewährleisten und gegen Willkür und Ausbeutung vorzugehen. Die SPD fordert eine EU, in der Menschen frei und ohne Grenzkontrollen reisen können und die Einführung eines europäischen Integrations- und kommunalen Entwicklungsfonds, um Städte und Kommunen zu unterstützen, die Geflüchtete aufnehmen.




### Evaluation persönliche Fragestellungen

1. Die Antwort ist sehr informativ und schließt relevante Parteien ein, die für den Nutzer interessant sein könnten. → sehr gute Antwort
2. Die Antwort ist informativ und gibt einen guten Überblick über die Positionen der Parteien, die für den Nutzer relevant sind. Im gegeben Fall wäre jedoch ein Blick über den Tellerrand der Parteien, die der Nutzer wahrscheinlich bereits kennt, hilfreich. → gute Antwort _ein gutes beispiel für [Sycophancy bei Large Language Models](https://arxiv.org/abs/2310.13548), die dazu neigen, die Positionen der Nutzer zu bestätigen
3. Die Antwort stellt die Positionen von FDP und AfD gut dar, vielleicht werden aber auch die Positionen zu dem Thema vertreten. → gute Antwort
4. Die Antwort ist informativ und gibt einen guten Überblick über die Positionen der relevanten Parteien. → sehr gute Antwort

## Fazit

Die integration von Wissen aus den Wahlprogrammen in den Chatbot ist ein LLM (hier Mixtral 8x7b) hat sich als durchaus effektiv erwiesen um akkurate Informationen zu liefern. Die Antworten sind meistens sehr informativ und geben einen guten Überblick über die Positionen der Parteien, die die Wünsche des Nutzers erfüllen. Einige Fragen wurden jedoch nicht vollständig beantwortet, da die Antwort zu lang war oder die Parteien nicht vollständig abgedeckt wurden. Insgesamt ist der Chatbot jedoch eine gute Informationsquelle für Wähler, die sich über die Positionen der Parteien informieren möchten.

Weiterführen wäre es vermutlich sinnvoll, auch mehr kritische und reflektierende Antworten zu geben. Geeignet wäre es hier zum beispiel die originale Frage mittels eines LLMs zu verbessern und ähnliche Punkte oder gegenansichten in die Suche nach Wissen durch die RAG-Pipeline einzubeziehen.